In [528]:
import pandas as pd
import numpy as np
import math

columns = 'Weather Parents Cash Exam Decision'            # Input Columns
data = '''1 sunny visit rich yes cinema
2 sunny no-visit rich no tennis
3 windy visit rich no cinema
4 rainy visit poor yes cinema
5 rainy no-visit rich no stay-in
6 rainy visit poor no cinema
7 windy no-visit poor yes cinema
8 windy no-visit rich yes shopping
9 windy visit rich no cinema
10 sunny no-visit rich no tennis
11 sunny no-visit poor yes tennis'''                # Input Data
def checkint(x):                                    # Check if data is a number
    try:
        int(x)
        return True
    except: return False
s = columns.split()
x = data.split()
li = []
t = []
for i in x:
    if not checkint(i): t += [i]
    else:
        if len(t)>0: li += [t]
        t = []                                      # Reset after appended once
if len(t)>0: li += [t]
df = pd.DataFrame(li, columns=s)
df

,Weather,Parents,Cash,Exam,Decision
0,sunny,visit,rich,yes,cinema
1,sunny,no-visit,rich,no,tennis
2,windy,visit,rich,no,cinema
3,rainy,visit,poor,yes,cinema
4,rainy,no-visit,rich,no,stay-in
5,rainy,visit,poor,no,cinema
6,windy,no-visit,poor,yes,cinema
7,windy,no-visit,rich,yes,shopping
8,windy,visit,rich,no,cinema
9,sunny,no-visit,rich,no,tennis


In [529]:
def log(x): return math.log2(x)                     # Defining for easy calling
def entropy(x, y=None):                             # Heart of the ID3 Algorithm, uses default value to check if single series passed
    if y is None:
        ent, tot = 0, x.count()
        entform = []
        for i in x.unique():
            num = list(x).count(i)
            ent -= num/tot*log(num/tot)
            entform += [f'-({num}/{tot})*log({num}/{tot})']
        print(f'Entropy of Data = {ent:.4f} = \t{' '.join(entform)}')
        return ent
    else:                                           # If two series passed, calculate entropy based on ID3 Algorithm
        ents = []
        for i in x.unique():
            inds=np.where(x == i)[0]
            den = list(x).count(i)
            vals = y[inds]
            nums = [list(vals).count(i) for i in vals.unique()]
            entform = []
            for nn in nums: entform += [f'-({nn}/{den})*log({nn}/{den})']
            ents += [sum([-i*log(i) for i in [i/den for i in nums]])]
            print(f"Entropy for '{i}' = {ents[-1]:.4f} = \t{' '.join(entform)}")
        return ents

In [530]:
y = df.iloc[:, -1]
ES = entropy(y)
print()
for v in range(df.shape[1]-1):                      # Loop to calculate Entropy and IG of every column
    x = df.iloc[:, v]
    print(f"--------- FOR '{x.name.upper()}' ---------")
    ents = entropy(x,y)
    fracs = [list(x).count(i) for i in x.unique()]
    infogain = ES                                   # Starting with es for easy subtraction
    igform = []
    for j,i in enumerate(fracs):
        infogain -= i/y.count() * ents[j]
        if ents[j]!=0: igform += [f'({i}/{y.count()})*{ents[j]:.2f}']
    print(f"\nInfo. Gain for '{x.name}' = \t{infogain:.4f} = \t{ES:.3f} - [{' + '.join(igform)}]\n")

Entropy of Data = 1.6172 = 	-(6/11)*log(6/11) -(3/11)*log(3/11) -(1/11)*log(1/11) -(1/11)*log(1/11)

--------- FOR 'WEATHER' ---------
Entropy for 'sunny' = 0.8113 = 	-(1/4)*log(1/4) -(3/4)*log(3/4)
Entropy for 'windy' = 0.8113 = 	-(3/4)*log(3/4) -(1/4)*log(1/4)
Entropy for 'rainy' = 0.9183 = 	-(2/3)*log(2/3) -(1/3)*log(1/3)

Info. Gain for 'Weather' = 	0.7767 = 	1.617 - [(4/11)*0.81 + (4/11)*0.81 + (3/11)*0.92]

--------- FOR 'PARENTS' ---------
Entropy for 'visit' = 0.0000 = 	-(5/5)*log(5/5)
Entropy for 'no-visit' = 1.7925 = 	-(3/6)*log(3/6) -(1/6)*log(1/6) -(1/6)*log(1/6) -(1/6)*log(1/6)

Info. Gain for 'Parents' = 	0.6395 = 	1.617 - [(6/11)*1.79]

--------- FOR 'CASH' ---------
Entropy for 'rich' = 1.8424 = 	-(3/7)*log(3/7) -(2/7)*log(2/7) -(1/7)*log(1/7) -(1/7)*log(1/7)
Entropy for 'poor' = 0.8113 = 	-(3/4)*log(3/4) -(1/4)*log(1/4)

Info. Gain for 'Cash' = 	0.1498 = 	1.617 - [(7/11)*1.84 + (4/11)*0.81]

--------- FOR 'EXAM' ---------
Entropy for 'yes' = 1.3710 = 	-(3/5)*log(3/5) -